In [49]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV,cross_val_score,KFold,RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report,accuracy_score,roc_curve,f1_score,recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier

In [3]:
path = "../data/interim/"

In [ ]:
# def classifica_texto(texto,coluna_texto,coluna_classificacao):
#     vetorizarResenha = CountVectorizer(lowercase=False, max_features=50)
#     bag_of_words_Resenha = vetorizarResenha.fit_transform(texto[coluna_texto])
#     treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words_Resenha,
#                                                               texto[coluna_classificacao],
#                                                               random_state = 42)

In [70]:
scorer = ["recall","accuracy","f1"]
def cross_score(data_X,data_Y, kfold,estimator,estimator_name,score=[],export=False):
    recall = []
    acc  = []
    f1 = []
    std_acc=[]
    std_recall=[]
    std_f1=[]
    print("model name: ",estimator_name)
    for sc in score:
        cv = cross_val_score(estimator,data_X,data_Y, scoring=sc, cv=kfold)
        mean = cv.mean()
        std = cv.std()
        
        print("Array: ")
        print("--------> "+sc + " : [ "+str(cv)+" ]\n")
        print(sc + " : "+ str(round(mean,3)) + " std : "+ str(round(std,3)))
        print("Intervalo :  [{:.2f}% ~ {:.2f}%]".format((mean - 2*std)*100, (mean + 2*std)*100))
        print()
        
        if sc == "recall":
            recall.append(round(mean,3))
            std_recall.append(round(std,3))
        elif sc == "f1":
            f1.append(round(mean,3))
            std_f1.append(round(std,3))
        else:
            acc.append(round(mean,3))
            std_acc.append(round(std,3))
        print("-"*30)
    if export is True:
        df_ = pd.DataFrame(data=zip(recall,f1,acc,std_acc,std_recall,std_f1), index=[estimator_name],\
                           columns=['recall','f1','accuracy',"std_acc","std_recall","std_f1"])
        return df_

In [24]:
def make_data_for_machine_learning_tfidf(df,col,label):
    vetorizarResenha = TfidfVectorizer(lowercase=False, max_features=50)
    bag_of_words_ = vetorizarResenha.fit_transform(df[col])
    X = pd.DataFrame(bag_of_words_.toarray(),columns=vetorizarResenha.get_feature_names_out())
    x_train, x_test, y_train, y_test = train_test_split(X,
                                                              df[label],
                                                            test_size=0.3,
                                                              random_state = 42)
    return x_train, x_test, y_train, y_test

## No stopWords

### Lemma

In [6]:
df_test = pd.read_csv(path+"no_stopWords_lemma.csv")
df_test.head()

,text,category,label,site
0,Kát abr dizer ir coloc expuls Mold reclam sen ...,politica,fake,https://ceticismopolitico.com
1,dr ray peit bolsonar chamar conserv fak entrev...,politica,fake,https://ceticismopolitico.com
2,reinald azeved desmascar políci feder ferrenh ...,politica,fake,https://afolhabrasil.com.br
3,relatóri assustr bnd mostr dinh públic Brasil ...,politica,fake,https://www.diariodobrasil.org
4,radial americ fal Sobr pt vend ilusãr pov Bras...,politica,fake,https://www.diariodobrasil.org


In [25]:
 x_train,x_test,y_train,y_test= make_data_for_machine_learning_tfidf(df_test,"text","label")

In [26]:
print("treino _ x: {}".format(x_train.shape))
print("treino _ y : {}".format(y_train.shape))
print("test _ x : {}".format(x_test.shape))
print("test _ y: {}".format(y_test.shape))     

treino _ x: (5040, 50)
treino _ y : (5040,)
test _ x : (2160, 50)
test _ y: (2160,)


#### Fazendo baseline

In [43]:
dummy_clf = DummyClassifier()

In [44]:
dummy_clf.fit(x_train,y_train)

DummyClassifier()

In [46]:
cross_score(x_test,y_test,3,dummy_clf,"DummyClassifier",scorer)

Array: 
--------> recall : [ [0. 0. 0.] ]

recall : 0.0 std : 0.0
Intervalo :  [0.00% ~ 0.00%]

Array: 
--------> accuracy : [ [0.50416667 0.50277778 0.50277778] ]

accuracy : 0.503 std : 0.001
Intervalo :  [50.19% ~ 50.46%]

Array: 
--------> f1 : [ [0. 0. 0.] ]

f1 : 0.0 std : 0.0
Intervalo :  [0.00% ~ 0.00%]



baseline acc 50%

In [47]:
#carregando os modelos

In [51]:
random_florest = RandomForestClassifier()
reg_logistic = LogisticRegression()
xgb = XGBClassifier()
tree = DecisionTreeClassifier()
ada = AdaBoostClassifier()
gbm = GradientBoostingClassifier()
gnb = GaussianNB()

In [53]:
models = [random_florest,reg_logistic,xgb,tree,ada,gbm,gnb]

In [72]:
result=[]
for model in models:
    aux = cross_score(x_train,y_train,3,model,model.__class__.__name__,scorer,export=True)
    result.append(aux)

model name:  RandomForestClassifier
Array: 
--------> recall : [ [0.94774347 0.93230404 0.9311981 ] ]

recall : 0.937 std : 0.008
Intervalo :  [92.20% ~ 95.22%]

------------------------------
Array: 
--------> accuracy : [ [0.94107143 0.94107143 0.93452381] ]

accuracy : 0.939 std : 0.003
Intervalo :  [93.27% ~ 94.51%]

------------------------------
Array: 
--------> f1 : [ [0.94250148 0.93437688 0.93738819] ]

f1 : 0.938 std : 0.003
Intervalo :  [93.14% ~ 94.48%]

------------------------------
model name:  LogisticRegression
Array: 
--------> recall : [ [0.9192399  0.90617577 0.89561091] ]

recall : 0.907 std : 0.01
Intervalo :  [88.77% ~ 92.63%]

------------------------------
Array: 
--------> accuracy : [ [0.90119048 0.90952381 0.89642857] ]

accuracy : 0.902 std : 0.005
Intervalo :  [89.16% ~ 91.32%]

------------------------------
Array: 
--------> f1 : [ [0.90315053 0.90941597 0.89667458] ]

f1 : 0.903 std : 0.005
Intervalo :  [89.27% ~ 91.35%]

------------------------------

In [73]:
resultado = pd.concat(result)

In [74]:
resultado

,recall,f1,accuracy,std_acc,std_recall,std_f1
RandomForestClassifier,0.937,0.938,0.939,0.003,0.008,0.003
LogisticRegression,0.907,0.903,0.902,0.005,0.010,0.005
XGBClassifier,0.942,0.941,0.941,0.005,0.010,0.005
DecisionTreeClassifier,0.877,0.877,0.874,0.002,0.014,0.002
AdaBoostClassifier,0.929,0.933,0.933,0.003,0.012,0.004
GradientBoostingClassifier,0.938,0.942,0.942,0.002,0.007,0.002
GaussianNB,0.856,0.841,0.838,0.011,0.007,0.009


Após análise dos dataframe acima iremos escolher os :
* RandomForest
* LogisticRegression
* GradientBoostingClassifier

In [75]:
random_florest.fit(x_train,y_train)
reg_logistic.fit(x_train,y_train)
gbm.fit(x_train,y_train)

GradientBoostingClassifier()

In [76]:
models = [random_florest,reg_logistic,gbm]

In [77]:
result=[]
for model in models:
    aux = cross_score(x_test,y_test,3,model,model.__class__.__name__,scorer,export=True)
    result.append(aux)

model name:  RandomForestClassifier
Array: 
--------> recall : [ [0.91316527 0.93575419 0.94972067] ]

recall : 0.933 std : 0.015
Intervalo :  [90.28% ~ 96.30%]

------------------------------
Array: 
--------> accuracy : [ [0.91666667 0.9375     0.94583333] ]

accuracy : 0.933 std : 0.012
Intervalo :  [90.88% ~ 95.79%]

------------------------------
Array: 
--------> f1 : [ [0.91830986 0.94036061 0.94329184] ]

f1 : 0.934 std : 0.011
Intervalo :  [91.17% ~ 95.63%]

------------------------------
model name:  LogisticRegression
Array: 
--------> recall : [ [0.89355742 0.88826816 0.91620112] ]

recall : 0.899 std : 0.012
Intervalo :  [87.51% ~ 92.36%]

------------------------------
Array: 
--------> accuracy : [ [0.8875     0.88472222 0.90694444] ]

accuracy : 0.893 std : 0.01
Intervalo :  [87.33% ~ 91.28%]

------------------------------
Array: 
--------> f1 : [ [0.88734353 0.88456189 0.90733057] ]

f1 : 0.893 std : 0.01
Intervalo :  [87.28% ~ 91.34%]

------------------------------


In [78]:
resultado = pd.concat(result)

In [79]:
resultado

,recall,f1,accuracy,std_acc,std_recall,std_f1
RandomForestClassifier,0.933,0.934,0.933,0.012,0.015,0.011
LogisticRegression,0.899,0.893,0.893,0.010,0.012,0.010
GradientBoostingClassifier,0.935,0.934,0.935,0.009,0.020,0.009


# Faltar tunar os hyperparametros e aplicar para os outros 3 datasets.
## faltar usar o kfold, hold-out, usar as funções do sklearn.